<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-dataset" data-toc-modified-id="Import-dataset-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import dataset</a></span></li><li><span><a href="#Embeddings-CTNet" data-toc-modified-id="Embeddings-CTNet-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Embeddings CTNet</a></span></li><li><span><a href="#Modificar-modelo-para-sacar-cosas" data-toc-modified-id="Modificar-modelo-para-sacar-cosas-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Modificar modelo para sacar cosas</a></span></li><li><span><a href="#One-graph-embedding" data-toc-modified-id="One-graph-embedding-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>One graph embedding</a></span></li><li><span><a href="#All-graphs-embedding" data-toc-modified-id="All-graphs-embedding-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>All graphs embedding</a></span></li><li><span><a href="#DIGL" data-toc-modified-id="DIGL-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>DIGL</a></span></li></ul></div>

Para CTNet, MinCutNet, GapNet and DIGL (4 figuras de cada task): 
- [ ] Embeddings de un grafo solo *conv2* -> dispersión
    * Seleccionar grafo que tenga buen accuracy y cambie más la predicción respecto a baseline
    * Train vs test
- [ ] Embeddings de to los grafos (después del readout) pa plotearlo todos juntos y que se vea la separación entre clases
    * Train vs test
- [ ] Grafo con ejes con peso CT matrix en CTNet
    * Seleccionar grafo que tenga buen accuracy y cambie más la predicción respecto a baseline
    * Train vs test
- [ ] Grafo con ejes adjacency modificada en GapNet
    * Seleccionar grafo que tenga buen accuracy y cambie más la predicción respecto a baseline
    * Train vs test
- [X] Datos sobre grafos

In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import torch
from nets import CTNet, GAPNet

import torch.nn.functional as F
from torch_geometric.loader import DataLoader
from transform_features import FeatureDegree
from torch_geometric.datasets import TUDataset

import torch.nn as nn

In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [25]:
@torch.no_grad()
def test(modelo, loader, device):
    modelo.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        pred, mc_loss, o_loss = modelo(data.x, data.edge_index, data.batch)
        loss = F.nll_loss(pred, data.y.view(-1)) + mc_loss + o_loss
        correct += pred.max(dim=1)[1].eq(data.y.view(-1)).sum().item()

    return loss, correct / len(loader.dataset)


## Import dataset

In [33]:
"""dataset = TUDataset(root='data_colab/TUDataset',name="REDDIT-BINARY", pre_transform=FeatureDegree(), use_node_attr=True)
BATCH_SIZE = 64
num_of_centers = 420"""

dataset = TUDataset(root='data_colab/TUDataset',name="MUTAG")
BATCH_SIZE = 32
num_of_centers = 17

In [34]:
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}, {dataset.num_node_features}, {dataset.num_node_attributes}')
print(f'Number of classes: {dataset.num_classes}')

print()
datum = dataset[0]  # Get the first graph object.
print(datum)
print('=============================================================')
# Gather some statistics about the first graph.
print(f'Number of nodes: {datum.num_nodes}')
print(f'Number of edges: {datum.num_edges}')
print(f'Average node degree: {datum.num_edges / datum.num_nodes:.2f}')
print(f'Has isolated nodes: {datum.has_isolated_nodes()}')
print(f'Has self-loops: {datum.has_self_loops()}')
print(f'Is undirected: {datum.is_undirected()}')


print(dataset)

Dataset: MUTAG(188):
Number of graphs: 188
Number of features: 7, 7, 0
Number of classes: 2

Data(edge_index=[2, 38], x=[17, 7], edge_attr=[38, 4], y=[1])
Number of nodes: 17
Number of edges: 38
Average node degree: 2.24
Has isolated nodes: False
Has self-loops: False
Is undirected: True
MUTAG(188)


In [35]:
loader =  DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

## Embeddings CTNet

**Probar modelo guardaddo**

In [29]:
modelito = "models/REDDIT-BINARY_CTNet_iter0.pth"
modelito = "models/MUTAG_CTNet_iter0.pth"

In [ ]:
#Epoch: 059, Train Loss: 165.455, Train Acc: 0.711, Test Loss: 150.841, Test Acc: 0.750
model =  CTNet(dataset.num_features, dataset.num_classes, k_centers=num_of_centers).to(device)
model.load_state_dict(torch.load(modelito, map_location=torch.device(device)))
model.eval()

In [ ]:
loss, acc = test(model, loader, device)

In [ ]:
loss, acc

## Modificar modelo para sacar cosas

In [36]:
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.nn import DenseGraphConv
from torch_geometric.utils import to_dense_batch, to_dense_adj
from CT_layer import dense_CT_rewiring
from MinCut_Layer import dense_mincut_pool

In [40]:
class CTNet_embedding(torch.nn.Module):
    def __init__(self, in_channels, out_channels, k_centers, hidden_channels=32):
        super(CTNet_embedding, self).__init__()
    
        self.lin1 = Linear(in_channels, hidden_channels)
        num_of_centers1 =  k_centers # k1 #order of number of nodes
        self.pool1 = Linear(hidden_channels, num_of_centers1)
        #self.CT = CTLayer()
        self.conv1 = DenseGraphConv(hidden_channels, hidden_channels)
        num_of_centers2 =  16 # k2 #mincut 
        self.pool2 = Linear(hidden_channels, num_of_centers2)
        #self.MinCut = MinCutLayer()
        self.conv2 = DenseGraphConv(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, hidden_channels) # MLPs towards out 
        self.lin3 = Linear(hidden_channels, out_channels)
 

    def forward(self, x, edge_index, batch):    # x torch.Size([N, N]),  data.batch  torch.Size([661])  
        # Make all adjacencies of size NxN 
        adj = to_dense_adj(edge_index, batch) # adj torch.Size(B, N, N])
        # Make all x_i of size N=MAX(N1,...,N20), e.g. N=40:
        x, mask = to_dense_batch(x, batch) # x torch.Size([20, N, 32]) ; mask torch.Size([20, N]) batch_size=20

        x = self.lin1(x)
        # First mincut pool for computing Fiedler adn rewire 
        s1  = self.pool1(x)

        if torch.isnan(adj).any():
            print("adj nan")
        if torch.isnan(x).any():
            print("x nan")
        
        # CT REWIRING
        adj, CT_loss, ortho_loss1 = dense_CT_rewiring(x, adj, s1, mask) # out: x torch.Size([20, N, F'=32]),  adj torch.Size([20, N, N])

        # CONV1: Now on x and rewired adj: 
        x = self.conv1(x, adj) #out: x torch.Size([20, N, F'=32])

        # MLP of k=16 outputs s
        s2 = self.pool2(x) # s torch.Size([20, N, k])
        
        # MINCUT_POOL
        x, adj, mincut_loss2, ortho_loss2 = dense_mincut_pool(x, adj, s2, mask) # out x torch.Size([20, k=16, F'=32]),  adj torch.Size([20, k2=16, k2=16])

        # CONV2: Now on coarsened x and adj: 
        x = self.conv2(x, adj) #out x torch.Size([20, 16, 32])
        conv2 = x.clone()
        
        # Readout for each of the 20 graphs
        x = x.sum(dim=1) # x torch.Size([20, 32])

        # Final MLP for graph classification: hidden channels = 32
        x = F.relu(self.lin2(x)) # x torch.Size([20, 32])
        x = self.lin3(x) #x torch.Size([20, 2])
        print(x.shape)
        
        CT_loss = CT_loss + ortho_loss1
        mincut_loss = mincut_loss2 + ortho_loss2
        return F.log_softmax(x, dim=-1), CT_loss, mincut_loss, conv2


In [41]:
model =  CTNet_embedding(dataset.num_features, dataset.num_classes, k_centers=num_of_centers).to(device)
model.load_state_dict(torch.load(modelito, map_location=torch.device(device)))
model.eval()

CTNet_embedding(
  (lin1): Linear(in_features=7, out_features=32, bias=True)
  (pool1): Linear(in_features=32, out_features=17, bias=True)
  (conv1): DenseGraphConv(32, 32)
  (pool2): Linear(in_features=32, out_features=16, bias=True)
  (conv2): DenseGraphConv(32, 32)
  (lin2): Linear(in_features=32, out_features=32, bias=True)
  (lin3): Linear(in_features=32, out_features=2, bias=True)
)

**Test modificado que coja los embeddings**

In [44]:
@torch.no_grad()
def test_embedd(modelo, loader, device):
    modelo.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        pred, mc_loss, o_loss, etc = modelo(data.x, data.edge_index, data.batch)
        loss = F.nll_loss(pred, data.y.view(-1)) + mc_loss + o_loss
        correct += pred.max(dim=1)[1].eq(data.y.view(-1)).sum().item()
        print(etc.shape)
    return loss, correct / len(loader.dataset)

In [45]:
loss, acc = test_embedd(model, loader, device)

torch.Size([32, 2])
torch.Size([32, 16, 32])
torch.Size([32, 2])
torch.Size([32, 16, 32])
torch.Size([32, 2])
torch.Size([32, 16, 32])
torch.Size([32, 2])
torch.Size([32, 16, 32])
torch.Size([32, 2])
torch.Size([32, 16, 32])
torch.Size([28, 2])
torch.Size([28, 16, 32])


In [46]:
loss, acc

(tensor(21.9119, device='cuda:0'), 0.8404255319148937)

## One graph embedding

## All graphs embedding

# Embeddings GapNet Laplacian

In [ ]:
test_loader =  DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
test_loader =  DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
model =  GAPNet(dataset.num_features, dataset.num_classes, derivative="laplacian",device=device)
model.load_state_dict(torch.load("models/REDDIT-BINARY_GAPNet_laplacian_iter0.pth", map_location=torch.device('cpu')))
model.eval()

# Statistical data about the graph

In [9]:
import networkx as nx
from torch_geometric import utils
import numpy as np
from tqdm.notebook import tqdm


nxdata = {
    'Assortativity':np.empty(len(dataset)),
    'Triangles':np.empty(len(dataset)),
    'Transitivity':np.empty(len(dataset)),
    'Clustering':np.empty(len(dataset))
}
for i, graph in enumerate(tqdm(dataset, desc='Datasets')):
    G = utils.to_networkx(graph, to_undirected=True)
    nxdata['Assortativity'][i] = nx.degree_assortativity_coefficient(G)
    nxdata['Triangles'][i] = np.sum(list(nx.triangles(G).values()))/3
    nxdata['Transitivity'][i] = nx.transitivity(G)
    nxdata['Clustering'][i] = nx.average_clustering(G)


Datasets:   0%|          | 0/2000 [00:00<?, ?it/s]

In [13]:
for k in nxdata:
    print(f"{k}: {nxdata[k].mean():.4f} +- {nxdata[k].std():.4f}")

Assortativity: -0.3635 +- 0.1747
Triangles: 24.8350 +- 41.9040
Transitivity: 0.0097 +- 0.0156
Clustering: 0.0476 +- 0.0639


# Stratify

In [ ]:
from sklearn.model_selection import train_test_split

def class_distrib(dataset):
    d = dict()
    for data in dataset:
        d[int(data.y.numpy())] = d.get(int(data.y.numpy()),0) + 1
    return d

def batch_class_distrib(batch):
    d = dict()
    for label in batch.y:
        d[int(label.numpy())] = d.get(int(label.numpy()),0) + 1
    return d

len(dataset.data.y)

In [ ]:
train_indices, val_indices = train_test_split(list(range(len(dataset.data.y))), test_size=0.2, stratify=dataset.data.y)
train_dataset = torch.utils.data.Subset(dataset, train_indices)
val_dataset = torch.utils.data.Subset(dataset, val_indices)

In [ ]:
class_distrib(dataset), class_distrib(train_dataset), class_distrib(val_dataset)

**Optional**

In [ ]:
from torch.utils.data import SequentialSampler

In [ ]:
class StratifiedSampler():
    """Stratified Sampling
    Provides equal representation of target classes in each batch
    """
    def __init__(self, class_vector, batch_size):
        """
        Arguments
        ---------
        class_vector : torch tensor
            a vector of class labels
        batch_size : integer
            batch_size
        """
        self.n_splits = int(class_vector.size(0) / batch_size)
        self.class_vector = class_vector

    def gen_sample_array(self):
        try:
            from sklearn.model_selection import StratifiedShuffleSplit
        except:
            print('Need scikit-learn for this functionality')
        import numpy as np
        
        s = StratifiedShuffleSplit(n_splits=self.n_splits, test_size=0.5)
        X = torch.randn(self.class_vector.size(0),2).numpy()
        y = self.class_vector.numpy()
        s.get_n_splits(X, y)

        train_index, test_index = next(s.split(X, y))
        return np.hstack([train_index, test_index])

    def __iter__(self):
        return iter(self.gen_sample_array())

    def __len__(self):
        return len(self.class_vector)

In [ ]:
sampler = StratifiedSampler(class_vector=dataset.data.y, batch_size=BATCH_SIZE)


In [ ]:
loader = DataLoader(dataset, batch_size=2, #train_dataset
                        shuffle=False, sampler=sampler)

## DIGL

In [ ]:
import scipy.sparse as sp
from torch_geometric.utils.convert import to_scipy_sparse_matrix, from_scipy_sparse_matrix
from torch_geometric.utils import dense_to_sparse

In [ ]:
from torch_geometric.transforms import BaseTransform

class DIGLedges(BaseTransform):
    def __init__(self, alpha:float, eps:float, use_edge_weigths = False):
        self.alpha = alpha
        self.eps = eps
        self.use_edge_weigths = use_edge_weigths

    def __call__(self, data):
        new_edges, new_weights = self.digl_edges(data.edge_index, self.alpha, self.eps)
        data.edge_index = new_edges
        
        if self.use_edge_weigths:
            data.edge_weight = new_weights
            
        return data
    
    
    def gdc(self, A: sp.csr_matrix, alpha: float, eps: float):
        N = A.shape[0]

        # Self-loops
        A_loop = sp.eye(N) + A

        # Symmetric transition matrix
        D_loop_vec = A_loop.sum(0).A1
        D_loop_vec_invsqrt = 1 / np.sqrt(D_loop_vec)
        D_loop_invsqrt = sp.diags(D_loop_vec_invsqrt)
        T_sym = D_loop_invsqrt @ A_loop @ D_loop_invsqrt

        # PPR-based diffusion
        S = alpha * sp.linalg.inv(sp.eye(N) - (1 - alpha) * T_sym)

        # Sparsify using threshold epsilon
        S_tilde = S.multiply(S >= eps)

        # Column-normalized transition matrix on graph S_tilde
        D_tilde_vec = S_tilde.sum(0).A1
        T_S = S_tilde / D_tilde_vec

        return T_S

    def digl_edges(self, edges, alpha, eps):
        A0 = sp.csr_matrix(to_scipy_sparse_matrix(edges))
        new_sp_matrix = sp.csr_matrix(gdc(A0, self.alpha, self.eps))
        new_edge_index, weights = from_scipy_sparse_matrix(new_sp_matrix)
        return new_edge_index, weights
    
    
    def __repr__(self) -> str:
        return f'{self.__class__.__name__}({self.alpha})'
    
    

In [ ]:
digl_data = TUDataset(root='data/digl/TUDataset',name='MUTAG', transform=DIGLedges(0.5, 0.02))

In [ ]:
digl_data[0].edge_index

In [ ]:
dataset[0].edge_index

In [ ]:
digl_data = TUDataset(root='data/digl/TUDataset',name='MUTAG')

In [ ]:
digl_data[0].edge_index